<a href="https://colab.research.google.com/github/gunisha30/codes/blob/master/Copy_of_audio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praat-parselmouth

In [ ]:
import moviepy.editor as mp
clip = mp.VideoFileClip('/content/drive/My Drive/awarathon/videos/sovitq3.mp4')
duration = math.floor(clip.duration)
image_time_interval = 2
print(duration)

40


In [ ]:
pip install pitch

In [ ]:
import pitch

p = pitch.find_pitch('/content/drive/My Drive/awarathon/George.wav')

print('pitch =', p)

In [ ]:
!sudo apt-get install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
!sudo apt-get install ffmpeg libav-tools
!sudo pip install pyaudio

In [ ]:
import librosa as l
import os
import numpy as np
os.chdir('/content/drive/My Drive/awarathon')
arr,sr=l.load('George.wav')
arr.size#/sr #length
#print(sr)
2871072/22050
u=22049*5
l=u-22049
arrnew=arr[l:u+1]
np.sum(arrnew)/22050

In [ ]:
#convert mp4 to wav 
import sys
import os
from moviepy.editor import *

# name is the path, if its in the root directory then its just the video name
def convert_to_audio(name):
  video = VideoFileClip(name)
  audio = video.audio
  audio.write_audiofile('snigdhaq3.wav')

#folder renaming: first-Nishkarsh second-Yash third-Vishesh

#os.chdir('/content/drive/My Drive/awarathon/videos')
name="snigdhaq3.mp4"
convert_to_audio(name)

In [ ]:
import parselmouth
from parselmouth.praat import call, run_file
import inspect as i
import sys
import scipy.io.wavfile
import math
import moviepy.editor as mp

def mysppaus(m,p,l):              #pause rate and articulation rate
    sound=p+"/"+m+".wav"          #entire path till audio file
    sourcerun="/content/drive/My Drive/awarathon/myspsolution.praat"                  #path till mysolution.praat
    path=p                        #path till the folder which has audio file
    try:
        objects= run_file(sourcerun, -20, 2, 1, "yes",sound,path, 80, 400, 0.01, capture_output=True) 
        z1=str(objects[1])
        z2=z1.strip().split()
        print(z2)
        print ("articulation rate=",z2[3])
        p=int(z2[1])                          #pause length in secs
        print(p)                            
        a=p/l                                 #ratio of pause length and total length
        print("rate of pause=",a)
        print ("pause percentage=",a*100)
    except:
        print ("Try again the sound of the audio was not clear")
    return;


audioname="Q2converted"
path1="/content/drive/My Drive/awarathon/first"
#path1='/content'
#length of video files 

clip = mp.VideoFileClip('/content/drive/My Drive/awarathon/first/Q2.mp4')
duration = math.floor(clip.duration) #in seconds
image_time_interval = 2
print('duration=',duration)
mysppaus(audioname,path1,duration)
#mysppaus(audioname,path1)

duration= 30
['96', '2', '3', '4', '25.1', '30', '0.8', '179.54', '42.03', '168.5', '73', '348', '158', '182', '0.85']
articulation rate= 4
2
rate of pause= 0.06666666666666667
pause percentage= 6.666666666666667


In [ ]:
pip install aubio

In [ ]:
import scipy.io.wavfile
import os
os.chdir('/content/drive/My Drive/awarathon')
rate, arr = scipy.io.wavfile.read("George.wav")
print(arr.shape[0])
print(arr.size)
length=arr.shape[0]/rate
print(length/60)
  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import youtube_dl

def download_from_vimeo(vid_link):
    ydl = youtube_dl.YoutubeDL({'outtmpl':'vid.mp4'})

    with ydl:
        result = ydl.extract_info(
            vid_link,
            download=True # We just want to extract the info
        )

    if 'entries' in result:
        # Can be a playlist or a list of videos
        video = result['entries'][0]
    else:
        # Just a video
        video = result

    print(video)

download_from_vimeo('https://player.vimeo.com/video/348632880')

In [ ]:
import math
f=200
p=69+(12*(math.log(f/440,2)))
print(p)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

#cap is a video capture object 

cap = cv2.VideoCapture("//content//drive//My Drive//awarathon//videos//gargiq2.mp4")
# rate, frame = cap.read()

while True:
    ret, frame = cap.read()
    cv2_imshow(frame)
    key=cv2.waitKey(0)
    if (key==27):
      break
#     #if ret is False:
#      #   break
# cv2.destroyAllWindows()

# roi = frame[269: 795, 537: 1416]
# rows, cols, _ = roi.shape
# gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
# gray_roi = cv2.GaussianBlur(gray_roi, (7, 7), 0)

In [ ]:
!pip install gaze_tracking

In [ ]:
import cv2

import numpy as np
import os

os.chdir('/content/drive/My Drive/eyecontact')

from gaze_tracking import GazeTracking

gaze = GazeTracking() 
input_stream = '/content/drive/My Drive/awarathon/videos/sovitq3.mp4'
cap = cv2.VideoCapture(input_stream) #cap is a video capture object. 
cap.open(input_stream) 
width = int(cap.get(3)) #this returns the height and width of frames in this video
height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID') #fourcc is videowriter object, XVID is a 4 byte code (FourCC code)
fps = cap.get(cv2.CAP_PROP_FPS) 
#fps = int(fps)
#out = cv2.VideoWriter('/content/drive/My Drive/awarathon/eyeoutput/darshanq6out.avi', fourcc, fps, (width,height)) #videowriter is used to process and save videos. video is saved in out.avi file
count = 0
vc = 0
res = []
while cap.isOpened():  #returns true if video capturing/accessing is successful 
    # We get a new frame from the inout
    flag, frame = cap.read()
    if not flag:
        break

    # We send this frame to GazeTracking to analyze it
    gaze.refresh(frame)

    frame = gaze.annotated_frame()
    text = ""
    num = -1

    if gaze.is_blinking():
        text = "Blinking"
        num = 0
    elif gaze.is_right():
        text = "Looking right"
        num = 2
    elif gaze.is_left():
        text = "Looking left"
        num = 3
    elif gaze.is_center():
        text = "Looking center"
        num = 1

    if num >= 0:
        res.append(num)


    #cv2.putText(frame, text, (90, 60), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147, 58, 31), 2) #used to write text on image 

    left_pupil = gaze.pupil_left_coords()
    right_pupil = gaze.pupil_right_coords()
    cv2.putText(frame, "Left pupil:  " + str(left_pupil), (90, 130), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
    cv2.putText(frame, "Right pupil: " + str(right_pupil), (90, 165), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)

    #out.write(frame)
    count += fps #count the total no of frames
    #cap.set(1, count) 
    vc += 1

    if cv2.waitKey(1) == 27:
        break

# from scipy import stats
#
# res = np.array(res)
# print(stats.describe(res))
# print(res)
from collections import Counter
dict = Counter(res)
print(dict)
print(sum(res)/len(res)) #get an average of all values(directions) in the res list 
cap.release()
cv2.destroyAllWindows()


Counter({0: 500, 1: 117})
0.18962722852512157


In [2]:
import cv2
import glob

import numpy as np
import os

os.chdir('/content/drive/My Drive/eyecontact')

from gaze_tracking import GazeTracking

f=glob.glob("/content/drive/My Drive/awarathon/tryauto/*.mp4")
for file in f:
  gaze = GazeTracking() 
  input_stream = file
  cap = cv2.VideoCapture(input_stream) #cap is a video capture object. 
  cap.open(input_stream) 
  width = int(cap.get(3)) #this returns the height and width of frames in this video
  height = int(cap.get(4))
  fourcc = cv2.VideoWriter_fourcc(*'XVID') #fourcc is videowriter object, XVID is a 4 byte code (FourCC code)
  fps = cap.get(cv2.CAP_PROP_FPS) 
  #fps = int(fps)
  #out = cv2.VideoWriter('/content/drive/My Drive/awarathon/eyeoutput/darshanq6out.avi', fourcc, fps, (width,height)) #videowriter is used to process and save videos. video is saved in out.avi file
  count = 0
  vc = 0
  res = []
  while cap.isOpened():  #returns true if video capturing/accessing is successful 
      # We get a new frame from the inout
      flag, frame = cap.read()
      if not flag:
          break

      # We send this frame to GazeTracking to analyze it
      gaze.refresh(frame)

      frame = gaze.annotated_frame()
      text = ""
      num = -1

      if gaze.is_blinking():
          text = "Blinking"
          num = 0
      elif gaze.is_right():
          text = "Looking right"
          num = 2
      elif gaze.is_left():
          text = "Looking left"
          num = 3
      elif gaze.is_center():
          text = "Looking center"
          num = 1

      if num >= 0:
          res.append(num)


      #cv2.putText(frame, text, (90, 60), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147, 58, 31), 2) #used to write text on image 

      left_pupil = gaze.pupil_left_coords()
      right_pupil = gaze.pupil_right_coords()
      cv2.putText(frame, "Left pupil:  " + str(left_pupil), (90, 130), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
      cv2.putText(frame, "Right pupil: " + str(right_pupil), (90, 165), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)

      #out.write(frame)
      count += fps #count the total no of frames
      #cap.set(1, count) 
      vc += 1

      if cv2.waitKey(1) == 27:
          break

  from collections import Counter
  dict = Counter(res)
  print(dict)
  print(sum(res)/len(res)) #get an average of all values(directions) in the res list 
  cap.release()
  cv2.destroyAllWindows()


Counter({3: 408, 1: 266, 0: 88, 2: 48})
1.9580246913580246


In [ ]:
x=[0.61,0.766,0.84,0.75,0.71,0.21,0.49,0.26,0.17,0.25,0.32,0.43,0.44,0.23,0.28,0.77,0.26,0.96,0.91,0.74,0.62,1,1.3]
b=5
a=0
for i in x:
  xnor=((b-a)*((i-min(x))/(max(x)-min(x))))+a
  print(int(xnor))